In [1]:
import requests

## Data Collection

In [4]:
subreddit = 'https://api.pushshift.io/reddit/search/submission/?subreddit=geegees'
post_data = requests.get(subreddit)

In [8]:
post_data.json()['data'][0]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'Slippuree',
 'author_flair_css_class': 'phys',
 'author_flair_richtext': [{'e': 'text', 't': 'Physics'}],
 'author_flair_template_id': 'f73409d8-2ae6-11e9-bfdc-0e42cc90da4c',
 'author_flair_text': 'Physics',
 'author_flair_text_color': 'dark',
 'author_flair_type': 'richtext',
 'author_fullname': 't2_xv3o2',
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1604269813,
 'domain': 'self.geegees',
 'full_link': 'https://www.reddit.com/r/geegees/comments/jmbj59/looking_for_laptop_recommendations/',
 'gildings': {},
 'id': 'jmbj59',
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locke